# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [32]:
import pandas as pd

clients = pd.read_csv('clients.csv')
clients.info()
clients.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Информация о клиентах. Размер таблицы - 21525 строк, 12 столбцов. Имеются пропущенные значения в столбцах <b>'days_employed'</b> и <b>'total_income'</b>. В столбце <b>'days_employed'</b> часто встречаются отрицательные значения, что представляется невозможным в реальной жизни, очевидно - это технологическая ошибка. В столбце <b>'education'</b> имеются данные типа 'object' в разных регистрах.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [82]:
cols_with_missing = [col for col in clients.columns if clients[col].isnull().any() == True]
print('Столбцы, имеющие пропущенные значения: {}'.format(cols_with_missing))

Столбцы, имеющие пропущенные значения: ['days_employed', 'total_income']


Два столбца, которые имеют пропущенные значения напрямую связаны. Количество дней в статусе "занятого" и "общий доход". Соответственно, если количество дней в статусе "занятого" является пропущенным значением, то и "общий доход" также будет являться пропущенным значением. Проверим.

In [83]:
missing_clients = clients[(clients['total_income'].isnull() == True) & (clients['days_employed'].isnull() == True)]
missing_clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Это действительно так. Пропуски в данных могли оказаться по трем причинам:
- Клиент безработный
- Клиент не указал эти данные
- Данные были утеряны

In [84]:
print('Количество клиентов с пропущенными значениями: {}'.format(missing_clients.shape[0]))

Количество клиентов с пропущенными значениями: 2174


In [85]:
print('Статусы клиентов с пропущенными значениями:')
print(missing_clients.groupby('income_type')['income_type'].count())

Статусы клиентов с пропущенными значениями:
income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64


Первое, на что могут указывать такие пропущенные значения - безработный клиент. Но, во входных данных и так имеется отдельный "статус" клиента, указывающий на его безработность - "безработный". В таком случае можно отклонить эту теорию.

In [86]:
print(clients['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


По всей видимости, данные были <b>утеряны</b>.
При условии, что "статус" клиента официально подтвержден, в случае, когда клиент не указал данные или они были утеряны, их можно восстановить с помощью нахождения <b>среднего значения</b> или <b>медианы</b> для каждой группы клиентов.
Рассмотрим обе величины.

In [87]:
print('Среднее значение "общего дохода" для каждой группы клиентов: ', clients.groupby('income_type')['total_income'].mean())
print('-----------------------------------------------------')
print('Медиана "общего дохода" для каждой группы клиентов: ', clients.groupby('income_type')['total_income'].median())

Среднее значение "общего дохода" для каждой группы клиентов:  income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64
-----------------------------------------------------
Медиана "общего дохода" для каждой группы клиентов:  income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


Обратим внимание, что именно в тех группах клиентов, где пропущены значения, среднее значение <b>больше</b>, чем медиана. <b>Кроме</b> "предприниматель" так как для этой группы лишь у одного клиента оказались пропущенные значения, они не повлияли на эти величины.
Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Это относится и к группам клиентов, где значения были пропущены. По данной причине выбираем <b>медиану</b> на замену пропущенных значений.

In [88]:
# создадим Series, где содержатся медианы для каждой группы клиентов
total_income_grouped_median = clients.groupby('income_type')['total_income'].median()
# заменим каждое пропущенное значение в столбце 'total_income' в соответствии со статусом
for income_type in missing_clients['income_type'].unique():
    clients.loc[(clients['income_type'] == income_type) & (clients['total_income'].isnull() == True), 'total_income'] = total_income_grouped_median[income_type]

# проверим наличие пропущенных значений в столбце 'total_income'
print(clients['total_income'].isnull().any())

False


Пропущенные значения в столбце <b>'total_income'</b> был заполнен по принципу медианы среди каждой группы клиентов относительно их статуса. Данные об "общем доходе" клиента скорее всего были утеряны, поэтому необходимо было их восстановить.
Следующий столбец, в котором необходимо избавиться от пропущенных значений - <b>'days_employed'</b>. Давайте посмотрим на медиану этого столбца для каждой группы клиентов.

In [89]:
# медиана стажа для каждой группы клиентов относительно их статуса
clients.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

Обратим внимание на два артефакта:
- Медиана для "безработных" и "пенсионеров" составляет около 366000 дней, что равно примерно 1000 лет.
- Медианы для остальных групп являются отрицательными

В <b>первую</b> очередь проверим, есть ли для "безработных" и "пенсионеров" более адекватные данные в столбце 'days_employed'.

In [90]:
print(clients[(clients['days_employed'] < 15000) & (clients['income_type'] == 'безработный')])
print(clients[(clients['days_employed'] < 15000) & (clients['income_type'] == 'пенсионер')])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


Допустим, что 15000 дней это средний стаж пенсионера, больше, чем 41 год, работать практически невозможно. Допускаем, что стаж у "пенсионеров" и "безработных" представлен в часах, а не в днях. Напишем функцию, которая переводит стаж из часов в дни. И функцию, которая изменит знак перед отрицательными значениями.

In [91]:
def from_hours_to_days_employed(days_employed):
    return (days_employed / 24)

def positive(days_employed):
    return -1*days_employed

# заменим значения в часах на значения в днях
clients.loc[(clients['income_type'] == 'пенсионер') | (clients['income_type'] == 'безработный'), 'days_employed'] = clients.loc[(clients['income_type'] == 'пенсионер') | (clients['income_type'] == 'безработный'), 'days_employed'].apply(from_hours_to_days_employed)
# сделаем отрицательные значения положительными
clients.loc[(clients['days_employed'] < 0), 'days_employed'] = clients.loc[clients['days_employed'] < 0].apply(positive)
clients.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь заменим пропущенные данные в столбце <b>'days_employed'</b> медианами для каждой группы клиентов соответственно статусу.

In [92]:
# создадим Series которое содержит медианы столбца 'days_employed'
days_employed_grouped_median = clients.groupby('income_type')['days_employed'].median()
# заменяем пропущенные значения на медианы
for income_type in missing_clients['income_type'].unique():
    clients.loc[(clients['income_type'] == income_type) & (clients['days_employed'].isnull() == True), 'days_employed'] = days_employed_grouped_median[income_type]

# проверяем изменения
clients.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь еще раз проверим наличие в данных столбцов с пропущенными значениями

In [93]:
cols_with_missing = [col for col in clients.columns if clients[col].isnull().any() == True]
print('Столбцы, имеющие пропущенные значения: {}'.format(cols_with_missing))

Столбцы, имеющие пропущенные значения: []


Пропущенные значения <b>отсутствуют</b>

Также, в таблицы были найдены строки, в которых столбец <b>'dob_years'</b> имеет нулевые значения (0). Причиной этому может быть потеря данных. Восстановим эти данные с помощью медианы возраста для каждой группы клиентов относительно их статуса.

In [94]:
clients[clients['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1574.202821,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
20462,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


In [95]:
dob_years_grouped_median = clients.groupby('income_type')['dob_years'].median()
dob_years_grouped_median

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

Все выглядит вполне логично, даже возраст пенсионера совпал идеально. Заменяем нулевые значения.

In [96]:
# заменяем нулевые значения медианами
for income_type in missing_clients['income_type'].unique():
    clients.loc[(clients['income_type'] == income_type) & (clients['dob_years'] == 0), 'dob_years'] = dob_years_grouped_median[income_type]

# проверяем, остались ли клиенты, у которых возраст - 0
clients[clients['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Клиентов с нулевым возрастом не осталось. Далее в таблице были обнаружены нелогичные данные в столбце 'children'. У некоторых клиентов этот показатель составляет -1. Такого быть не может, дети либо есть (1, 2, 3 и т.д.), либо их нет (0). Допустим, что значения -1 означает, что детей нет. Заменим эти значения на 0.

In [97]:
clients[clients['children'] == -1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46.0,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,902.084528,50.0,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,3174.456205,57.0,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,14582.827176,54.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,15217.221094,57.0,Среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,на покупку своего автомобиля
1363,-1,1195.264956,55.0,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,1461.303336,38.0,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,2539.761232,42.0,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,3045.290443,26.0,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,901.101738,41.0,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


In [98]:
# заменяем значения -1 на 0
clients['children'] = clients['children'].replace(-1, 0)
# проверяем, остались ли отрицательные значения для столбца 'children'
clients[clients['children'] == -1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Также, в таблице были найдены <b>совершенно нелогичные данные</b>, которые лучше не использовать для выводов. Например, 22-х летний клиент, который имеет статус "пенсионер". Или "стаж труда" составляет больше, чем возраст клиента, или означает, что клиент работал с 12, 14, 16 лет. Стаж труда принято официально считать с 18 лет. В связи с нахождением таких данных, введем критерии для отбора <b>"нелогичных"</b> данных, которые лучше не использовать для выводов:
- "пенсионеры" должны быть старше 45 лет (в условиях реального мира существуют пенсионные программы, например, для военнослужащих или других госслужащих, которые выходят на пенсию гораздно раньше обычных сотрудников)
- "стаж труда" должен соответствовать условию: "возраст" - "стаж труда" >= 18.

In [99]:
# пенсионеры младше 45 лет
clients[(clients['income_type'] == 'пенсионер') & (clients['dob_years'] < 45)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
157,0,14517.251167,38.0,среднее,1,женат / замужем,0,F,пенсионер,1,113560.650035,сделка с автомобилем
311,0,14759.113178,44.0,Среднее,1,женат / замужем,0,F,пенсионер,0,47457.979961,заняться высшим образованием
711,2,14160.579474,44.0,среднее,1,женат / замужем,0,M,пенсионер,0,169404.469473,на покупку автомобиля
751,0,16281.477669,41.0,среднее,1,женат / замужем,0,M,пенсионер,0,151898.693438,операции со своей недвижимостью
776,0,15222.356680,38.0,среднее,1,женат / замужем,0,F,пенсионер,0,73859.425084,покупка недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
20271,0,15993.010180,37.0,высшее,0,женат / замужем,0,F,пенсионер,0,188236.765814,на покупку автомобиля
20342,1,15358.290833,44.0,высшее,0,женат / замужем,0,M,пенсионер,0,81196.474337,недвижимость
20560,0,13891.416702,33.0,Среднее,1,женат / замужем,0,M,пенсионер,0,163411.879628,образование
20829,1,16591.942999,40.0,среднее,1,женат / замужем,0,F,пенсионер,0,39107.907534,получение образования


In [100]:
# клиенты, у которых "возраст" - "стаж труда" < 18 лет.
clients[(clients['dob_years'] - (clients['days_employed'] / 365) < 18)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
18,0,16678.380705,53.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
42,0,1257.496190,20.0,неоконченное высшее,2,Не женат / не замужем,4,F,сотрудник,0,82065.089021,получение образования
...,...,...,...,...,...,...,...,...,...,...,...,...
21485,0,15166.518713,59.0,высшее,0,женат / замужем,0,F,пенсионер,0,267000.734155,на покупку подержанного автомобиля
21505,0,14121.036100,53.0,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,16104.071420,62.0,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,15090.043922,59.0,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью


**Вывод**

В столбцах 'days_employed' и 'total_income' обнаружились пропущенные значения.
Причины появления пропусков в этих данных могут быть разными:
- клиент безработный;
- клиент не указал эти данные;
- данные были утеряны;

Пропуски в столбце 'total_income' были заполнены медианой "общего дохода" относительно каждой группы клиентов соответственно их статусу: Пенсионер, сотрудник и т.д.
Пропуски в столбце 'days_employed' также были заполнены медианой "стажа труда" относительно каждой группы клиентов соответственно их статусу.

### Замена типа данных

Проверим, какие столбцы имеют вещественный тип данных

In [101]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


Столбцы 'days_employed', 'total_income' и 'dob_years' имеют вещественный тип данных. Переведем эти данные в целочисленный тип с помощью метода <b>astype()</b>

In [102]:
clients[['days_employed','dob_years','total_income']] = clients[['days_employed',
                                                                 'dob_years','total_income']].astype('int')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для изменения типа данных был выбран метод <b>astype()</b>, а не <b>pd.to_numeric()</b>, так как данные уже имеют тип 'float64', а нам необходимо перевести их в тип 'int64', с чем успешно справляется данный метод.

### Обработка дубликатов

Найдем все столбцы, которые имеют тип данных 'object'

In [103]:
clients.info()
object_columns = ['education', 'family_status', 'gender', 'income_type']
print('Столбцы с типом данных "object": {}'.format(object_columns))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
Столбцы с типом данных "object": ['education', 'family_status', 'gender', 'income_type']


Проверим каждый столбец с помощью метода <b>value_couns()</b>. Столбец 'purpose' не был включен в список, так как цели кредита могут повторяться для любых клиентов.

In [104]:
for column in object_columns:
    print(clients[column].value_counts())
    print('----------------------------')

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
----------------------------
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
----------------------------
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
----------------------------
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент           

Столбец <b>'family_status'</b> не имеет дубликатов, но его строки мы все равно переведем в нижний регистр для удобства дальнейшей работы. Столбец <b>'gender'</b> не имеет дубликатов, но имеет одно значение "XNA", предположим, что этот клиент не определился с гендером. Столбец <b>'income_type'</b> также не имеет дубликатов, его мы изменять не будем. Все внимание переходит на столбец <b>'education'</b>, где имеется куча дубликатов, и это вызвано различным регистром букв, исправим это.

In [105]:
clients['family_status'] = clients['family_status'].str.lower() # устанавливаем нижний регистр для 'family_status'
clients['education'] = clients['education'].str.lower() # устанавливаем нижний регистр для 'education'
# проверяем наличие дубликатов
for column in object_columns:
    print(clients[column].value_counts())
    print('----------------------------')

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
----------------------------
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
----------------------------
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
----------------------------
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64
----------------------------


Теперь, когда все строки переведены в нижний регистр, можно воспользоваться методами <b>duplicated()</b> и <b>drop_duplicates()</b>, они могут показать нам полные дубликаты.

In [106]:
duplicated_rows = clients.duplicated()
# создадим список с индексами строк, которые выявил метод
duplicated_rows_index = duplicated_rows[duplicated_rows == True].index
print(duplicated_rows_index)
print('Количество дубликатов: {}'.format(duplicated_rows[duplicated_rows == True].count()))
# метод выявил дубликаты, удалим их
clients = clients.drop_duplicates().reset_index(drop=True)
# проверим наличие дубликатов
if clients.duplicated().all() == False:
    print('Дубликатов не существует')

Int64Index([ 2849,  3290,  4182,  4851,  5557,  6312,  7808,  7921,  7938,
             8583,  9238,  9528,  9604,  9627,  9855, 10462, 10697, 10864,
            10994, 11791, 12373, 12375, 12736, 13025, 13639, 13773, 13878,
            13942, 14097, 14432, 14728, 14832, 15091, 15188, 15273, 15991,
            16176, 16204, 16378, 16902, 16904, 17338, 17379, 17755, 17774,
            18328, 18349, 18428, 18521, 18563, 18755, 19041, 19184, 19295,
            19321, 19369, 19387, 19559, 19688, 19832, 19946, 20116, 20165,
            20187, 20297, 20662, 20702, 21032, 21132, 21281, 21415],
           dtype='int64')
Количество дубликатов: 71
Дубликатов не существует


Дубликаты <b>отсутствуют</b>

**Вывод**

Для поиска дубликатов были использованы два способа:
- метод value_counts(), который анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке.
- методы duplicated() и drop_duplicates()

Были использованы оба способа, так как изначально в строковых столбцах данные имели разный регистр, соответственно метод <b>duplicated()</b> не смог бы выявить полные дубликаты, поэтому изначально был использован метод <b>value_couns()</b> и данные в строковых столбцах были приведены в нижний регистр.

Дубликаты в столбце 'education' могли возникнуть по следующей причине: клиент или кредитный специалист заполнял форму с данными о клиенте, в поле "образование" каждый человек написал данные по разному "Среднее", "среднее", "высшее", "ВЫСШЕЕ" и так далее. Проблема в том, что, скорее всего, не было указано единой формы заполнения поля "образование".

### Лемматизация

In [147]:
from pymystem3 import Mystem
from itertools import chain
from collections import Counter

words = []

# Посчитаем количество каждого слова в "целях кредита" по всему DataFrame
for row in range(clients.shape[0]):
    words.append(m.lemmatize(clients['purpose'][row]))
    
words = list(chain.from_iterable(words))
print('Количество каждого слова в "целях кредита" в DataFrame:')
words_dict = dict(Counter(words))
words_dict = sorted(words_dict.items(), key=lambda x: x[1], reverse=True)
for i in words_dict:
    print('{}:'.format(i[0]), i[1])

Количество каждого слова в "целях кредита" в DataFrame:
 : 33570

: 21454
недвижимость: 6351
покупка: 5897
жилье: 4460
автомобиль: 4306
образование: 4013
с: 2918
операция: 2604
свадьба: 2324
свой: 2230
на: 2222
строительство: 1878
высокий: 1374
получение: 1314
коммерческий: 1311
для: 1289
жилой: 1230
сделка: 941
дополнительный: 906
заниматься: 904
проведение: 768
сыграть: 765
сдача: 651
семья: 638
собственный: 635
со: 627
ремонт: 607
подержанный: 486
подержать: 478
приобретение: 461
профильный: 436


Рассмотрим слова в данном списке. Выделяются следующие основные слова, которые могут находиться в "целях кредита":
- Недвижимость
- Жилье
- Автомобиль
- Образование
- Свадьба

Также, было обнаружено слово "коммерческий". Давайте посмотрим, в каких предложениях содержится данное слово.

Как видим, коммерческие цели идут совместно с недвижимостью, поэтому слова "коммерческий" будет достаточно, чтобы отнести эти цели к определенной группе. Добавим группу "коммерция". "Недвижимость" и "жилье" соединим в одну группу "жилье". Итоговый список целей кредитов:

- Жилье
- Автомобиль
- Образование
- Коммерция
- Свадьба

In [149]:
def purpose_stemmed(purpose):
    lemmas = m.lemmatize(purpose)
    if 'коммерческий' in lemmas:
        return 'коммерция'
    elif 'жилье' in lemmas:
        return 'жилье'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'недвижимость' in lemmas:
        return 'жилье'
        
clients['purpose_stemmed'] = clients.loc[:, 'purpose'].apply(purpose_stemmed)

# Проверяем новый столбец "purpose_stemmed"
print(clients['purpose_stemmed'])

0              жилье
1         автомобиль
2              жилье
3        образование
4            свадьба
            ...     
21449          жилье
21450     автомобиль
21451          жилье
21452     автомобиль
21453     автомобиль
Name: purpose_stemmed, Length: 21454, dtype: object


Проверяем новый столбец 'purpose_stemmed', который точнее характеризует цель кредита

In [150]:
clients['purpose_stemmed'].value_counts()

жилье          9500
автомобиль     4306
образование    4013
свадьба        2324
коммерция      1311
Name: purpose_stemmed, dtype: int64

Как видим, большинство кредит берется на жилье. Меньшая часть приходится на коммерцию.

**Вывод**

Лемматизация целей кредита была проведена следующим образом:
1) Импортирование лемматизатора

2) Создание 'Series' на основании целей кредита относительно каждого клиента с помощью лемматизатора (для уточнения целей кредита были взяты самые часто встречающиеся слова: "автомобиль", "жилье" и так далее. Проверка нахождения данных слов в лемматизированной версии "цели кредита".)

3) Добавление 'Series' к нашему 'DataFrame' в виде нового столбца 'purpose_stemmed'

Вывод: Большинство клиентов берут кредит на жилье, меньшинство берут кредит на коммерческие операции.

### Категоризация данных

В первую очередь, можем категоризовать данные по "целям кредита", так как это один из важных показателей

In [151]:
clients.groupby('purpose_stemmed').mean()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
purpose_stemmed,,,,,,,
автомобиль,0.536693,4659.627729,43.689039,0.820483,0.989549,0.093590,165222.227125
жилье,0.535053,4540.706105,43.336000,0.817053,0.967579,0.071895,166839.711263
коммерция,0.587338,4611.192982,43.543860,0.802441,0.986270,0.075515,165684.400458
образование,0.551209,4598.974084,43.609768,0.828059,0.952903,0.092200,162622.039870
свадьба,0.540017,4660.984940,43.372203,0.800344,1.000000,0.080034,163738.141997


Благодаря такой категоризации мы можем сделать следующие выводы:
- Клиенты, относящиеся к разным группам "целей кредитов" имеют практически схожие средние показатели по всем параметрам, кроме "возврата кредита в срок".
- Кредиты с целями "жилье" и "коммерция" наиболее чаще возвращаются в срок, чем другие

Также мы можем категоризовать клиентов по наличию детей, введем три группы:
- Бездетный
- Детный
- Многодетный

Напишем функцию, которая будет определять количество детей клиента и относить его к определенной группе.

In [152]:
def parent(children):
    if children == 0:
        return 'бездетный'
    elif 1 <= children <= 2:
        return 'детный'
    elif children >= 3:
        return 'многодетный'
    
# применим функцию к нашим данным и создадим новый столбец "parent"
clients['parent'] = clients['children'].apply(parent)


clients.groupby('parent').mean()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
parent,,,,,,,
бездетный,0.000000,5594.259726,46.440232,0.830881,1.122436,0.075258,163019.561748
детный,1.299125,2664.544315,37.760496,0.787755,0.700729,0.093003,169360.094752
многодетный,5.962719,2552.046053,37.493421,0.831140,0.478070,0.085526,175844.934211


Благодаря такой категоризации мы можем сделать следующие выводы:
- Бездетные клиенты в среднем имеют больше трудового стажа
- Бездетные клиенты, которые берут кредит, в среднем старше, чем клиенты с детьми
- Бездетные клиенты в среднем чаще возвращают кредит в срок
- Клиенты с 1 ребенком или 2 детьми в среднем возвращают кредит в срок реже, чем бездетные и многодетные клиенты
- Многодетные клиенты в среднем имеют больший доход, чем бездетные и клиенты с 1 или 2 детьми.

Следующую категоризацию мы можем провести по параметру "семейное положение".

In [153]:
clients.groupby('family_status').mean()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
family_status,,,,,,,
в разводе,0.461088,4725.255230,45.881172,0.797490,3.0,0.071130,167702.644351
вдовец / вдова,0.228363,9798.485923,56.816475,0.936392,2.0,0.065693,142590.553702
гражданский брак,0.513370,4233.559865,42.272224,0.837389,1.0,0.093471,164798.583715
женат / замужем,0.641786,4543.026663,43.715779,0.805090,0.0,0.075452,166801.413162
не женат / не замужем,0.288612,3509.185053,38.615658,0.807473,4.0,0.097509,166325.816726


Благодаря такой категоризации мы можем сделать следующие выводы:
- Вдовы и вдовцы в среднем имеют больший "стаж труда" чем клиенты с другим семейным положением
- Вдовы и вдовцы в среднем имеют выше "образование" чем клиенты с другим семейным положением
- Вдовы и вдовцы в среднем имеют меньше "детей" чем клиенты с другим семейным положением
- Вдовы и вдовцы в среднем старше чем клиенты с другим семейным положением
- Вдовы и вдовцы в среднем имеют меньше "общий доход" чем клиенты с другим семейным положением
- Вдовы и вдовцы в среднем чаще отдают кредит в срок чем клиенты с другим семейным положением
- Не женатые и не замужние клиенты в среднем младше чем клиенты с другим семейным положением
- Клиенты, находящиеся не замужем и не женаты, а также в гражданском браке, в среднем реже отдают кредит в срок, чем клиенты с другим семейным положением

Также мы можем категоризировать клиентов по уровню "общего дохода". Это является важнейшим показателем при работе скоринговой модели банка. Создадим функцию, которая будет относить клиентов в четыре разные группы:
- Богатый
- Обеспеченный
- Бедный

In [154]:
def total_income_group(total_income):
    if total_income < 80000:
        return 'бедный'
    elif 80000 <= total_income <= 300000:
        return 'обеспеченный'
    elif total_income > 300000:
        return 'богатый'
    
    
# Создадим новый столбец, который характеризует доход клиента
clients['income_group'] = clients['total_income'].apply(total_income_group)

clients.groupby('income_group').mean()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
income_group,,,,,,,
бедный,0.449912,6835.356766,46.748243,0.956942,0.920035,0.076450,63055.610281
богатый,0.606204,3623.304113,42.763992,0.566419,0.898854,0.071477,415548.084963
обеспеченный,0.548630,4385.618480,43.113196,0.820119,0.987115,0.082566,157501.764623


Благодаря такой категоризации мы можем сделать следующие выводы:
- Бедные клиенты в среднем имеют меньше детей
- Бедные клиенты в среднем имеют наибольший "стаж труда"
- Богатые клиенты в среднем имеют выше "образование"
- Богатые клиенты в среднем чаще отдают кредит срок, чем другие клиенты
- Обеспеченные клиенты в среднем реже других отдают кредит в срок

**Вывод**

Выводы представлены для каждой категоризации выше

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [155]:
clients.groupby('parent')[['debt']].mean().sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
parent,
детный,9.30%
многодетный,8.55%
бездетный,7.53%


**Вывод**

Зависимость есть. Результат говорит о том, что бездетные клиенты в среднем чаще отдают кредит в срок, чем многодетные клиенты. Многодетные клиенты в среднем чаще отдают кредит в срок, чем клиенты с 1 или 2 детьми.

Можно сделать следующие выводы:
- Бездетные клиенты не имеют дополнительных затрат на детей, поэтому им легче управлять своими финансами и возвращать кредит в срок.
- Многодетные клиенты имеют большую степень ответственности по жизни в целом, ведь у них больше 2 детей, а их надо обеспечивать и не залезать в долги перед банками.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [156]:
clients.groupby('family_status')[['debt']].mean().sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
family_status,
не женат / не замужем,9.75%
гражданский брак,9.35%
женат / замужем,7.55%
в разводе,7.11%
вдовец / вдова,6.57%


In [157]:
clients.groupby('family_status')[['debt']].mean().style.format({'debt': '{:.2%}'})

,debt
family_status,
в разводе,7.11%
вдовец / вдова,6.57%
гражданский брак,9.35%
женат / замужем,7.55%
не женат / не замужем,9.75%


**Вывод**

Зависимость есть. Вдовцы и вдовы в среднем отдают кредит в срок чаще, чем другие клиенты. Можем наблюдать следующую тенденцию: клиенты, которые в общем имели опыт настоящего официального брака, в среднем отдают кредит в срок чаще, чем клиенты, которые не имели такого опыта.

Можно сделать следующий вывод: брак придает людям больше ответственности и учит управлять финансами семьи в целом, благодаря чему такие клиенты чаще отдают кредит в срок, чем другие.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [158]:
clients.groupby('income_group')[['debt']].mean().sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
income_group,
обеспеченный,8.26%
бедный,7.64%
богатый,7.15%


**Вывод**

Размуеется, зависимость есть, ведь такой показатель, как "общий доход" является важнейшим показателем, когда речь идет о кредитах.

Можно сделать следующие выводы:
- Богатые клиенты в среднем отдают кредит в срок чаще, чем другие, так как такие клиенты лучше приспособлены к управлению финансами, они образованнее и имеют лучшее понятие о своих возможностях, когда думают брать кредит.
- Бедные люди в среднем отдают кредит в срок чаще, чем обеспеченные, так как бедные клиенты понимают, что они имеют ограниченные финансы, соответственно, такие клиенты чаще задумываются об управлении своим бюджетом, что способствует своевременным выплатам по кредиту.

- Как разные цели кредита влияют на его возврат в срок?

In [159]:
clients.groupby('purpose_stemmed')[['debt']].mean().sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
purpose_stemmed,
автомобиль,9.36%
образование,9.22%
свадьба,8.00%
коммерция,7.55%
жилье,7.19%


**Вывод**

Зависимость присутствует. Причем вполне объяснимая.

Можно сделать следующие выводы:
- Кредит с целью "автомобиль" в среднем отдается реже, чем другие, так как, когда идет речь об "автомобиле", необходимо учитывать дополнительные аспекты. Автомобиль может неожиданно сломаться, его ремонт может очень дорого обойтись, что поспособствует уменьшению финансов клиента. Автомобиль может попасть в ДТП, его ремонт также может дорого обойтись, что также поспособствует уменьшению финансов клиента.
- Кредит с целью "образование" в среднем отдается реже, чем другие, за исключением кредитов на "автомобиль", так как не всегда образование, полученное клиентом, окупает себя. Возможно, клиент получил повышение образования, но он никак не воспользовался своими знаниями и не увеличил свой доход. В итоге - кредит взят, доход не увеличен.
- Кредит с целью "жилье" в среднем отдается чаще, чем другие, так как это очень ответственный кредит, клиенты очень много раз думают перед тем, как действительно взять такой кредит, тем более данные кредиты обычно выдаются на долгий срок, что заставляет клиента еще раз подумать о своих возможностях.
- Кредит с целью "коммерция" в среднем отдается чаще, чем другие, за исключением кредитов на "жилье", так как клиенты, которые берут кредит с этой целью, заранее составляют бизнес-план и план возвращения кредита в кратчайшие сроки, ведь это в их же интересах. Это способствует своевременным выплатам по кредиту.
- Кредит с целью "свадьба" в среднем затисался в середине нашего рейтинга, возможно потому, что свадьба это иногда серьезное, обдуманное событие, а иногда очень даже спонтанное. Это и влияет на своевременность дальнейших выплат по кредиту.

## Шаг 4. Общий вывод

Во входных данных были найдены пропуски, а именно были пропущены значения "общего дохода" и "стажа труда" для некоторых клиентов. Эти пропуски могли возникнуть по причине утери данных, например, при их транспортировке. Пропущенные значения были заменены на медианы в соответствии со "статусами" клиентов.

В столбце "образование" нашлись дубликаты. Степень образования была написана в разных регистрах для разных клиентов. Проблема могла заключаться в том, что при оформлении заявки на кредит, данные об образовании были внесены клиентами по разному: "ВЫСШЕЕ", "высшее", "Среднее", "среднее". В форме заявки не был указан стандарт заполнения для поля "образования".


Можно сделать следующие основные выводы:
1) Кредиты с целью "жилье" в среднем возвращаются в срок чаще;

2) Богатые клиенты в среднем выпалчивают кредит в срок чаще;

3) Не женатые и не замужние клиенты в среднем выплачивают кредит в срок чаще;

4) Бездетные клиенты в среднем выплачивают кредит в срок чаще.

Картина <b>наилучшего</b> заемщика следующая:
- Кредит с целью "жилье";
- Доход заемщика выше 300000;
- Заемщик не замужем или не женат;
- Заемщик не имеет детей.